<a href="https://colab.research.google.com/github/ms3108/sentiment-analysis/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install kaggle

In [ ]:

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 94% 76.0M/80.9M [00:00<00:00, 131MB/s]
100% 80.9M/80.9M [00:00<00:00, 130MB/s]


In [ ]:
from zipfile import ZipFile
data_set = "/content/sentiment140.zip"
with ZipFile(data_set,'r') as zip:
  zip.extractall()
  print('done')

done


In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:

column_names =['target', 'id', 'date', 'flag', 'user', 'text']

twitter_data= pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=column_names,encoding='latin-1')

In [ ]:
twitter_data.shape

(1600000, 6)

In [ ]:
twitter_data.head()


,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
twitter_data.isnull().sum()

,0
target,0
id,0
date,0
flag,0
user,0
text,0


In [ ]:
twitter_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1600000 non-null  int64 
 1   id      1600000 non-null  int64 
 2   date    1600000 non-null  object
 3   flag    1600000 non-null  object
 4   user    1600000 non-null  object
 5   text    1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [ ]:
twitter_data['target'].value_counts()


,count
target,
0,800000
1,800000


In [ ]:
twitter_data.replace({'target':{4:1}},inplace=True)



*   1 positive tweet
*   0 negative tweet




In [ ]:
port_stem=PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content=re.sub('[^a-zA]','',content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=' '.join(stemmed_content)
  return stemmed_content


In [ ]:
twitter_data['stemmed_content']=twitter_data['text'].apply(stemming)


KeyboardInterrupt: 

In [ ]:
twitter_data.head()


,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoothttptwitpiccomyzlawwwthatsabummerous...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,isupsetthathecantupdatehisacebookbytextingitan...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,enichandivedmanytimesfortheballanagedtosaveher...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,mywholebodyfeelsitchyandlikeitsonfir
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclassnoitsnotbehavingatallimmadwhyam...


In [ ]:
print(twitter_data['stemmed_content'][0])

switchfoothttptwitpiccomyzlawwwthatsabummeroushouldagotavidarrofhirdaytodoit


In [ ]:
X=twitter_data['stemmed_content'].values
Y=twitter_data['target'].values

In [ ]:
# prompt: print(X)

print(X)
print(Y)


['switchfoothttptwitpiccomyzlawwwthatsabummeroushouldagotavidarrofhirdaytodoit'
 'isupsetthathecantupdatehisacebookbytextingitandmightcryasaresultchooltodayalsolah'
 'enichandivedmanytimesfortheballanagedtosaveherestgooutofbound' ...
 'areyoureadyforyourooakeoveraskmefordetail'
 'appythirthdaytomybooofallltimeupacamaruhakur'
 'happycharitytuesdaytheparksharitypeakingp']
[0 0 0 ... 1 1 1]


In [ ]:
X_Train,X_Test,Y_Train,Y_Test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

In [ ]:
vectorizer=TfidfVectorizer()
X_Train=vectorizer.fit_transform(X_Train)
X_Test=vectorizer.transform(X_Test)

In [ ]:
model = LogisticRegression(max_iter=1000)
model.fit(X_Train,Y_Train)

LogisticRegression(max_iter=1000)

In [ ]:
Prediction=model.predict(X_Train)
accuracyscore=accuracy_score(Y_Train,Prediction)
print(accuracyscore)


0.996528125


In [ ]:
testPrediction=model.predict(X_Test)
accuracyscoretest=accuracy_score(Y_Test,testPrediction)
print("model accuracy: ",accuracyscoretest)

model accuracy:  0.511165625


In [ ]:
import pickle


In [ ]:
filename='sentiment.sav'
pickle.dump(model,open(filename,'wb'))

In [ ]:
loaded_model=pickle.load(open(filename,'rb'))

In [ ]:
Xnew = X_Test[200]
prediction=loaded_model.predict(Xnew)
if prediction==1:
  print("positive")
else :
  print("negative")

positive
